In [17]:
import torch
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt

In [18]:
torch.cuda.is_available()

False

In [30]:
USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial
device = torch.device("cpu")
try:
    if USE_GPU and torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device('cpu')
except:
    if USE_GPU and torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100
print('using device:', device)

using device: mps


In [31]:
def plotImage(image):
    return plt.imshow(image.T)

In [32]:
BASE = "lung_colon_image_set/"

In [33]:
class LungColonDS(torchvision.datasets.DatasetFolder):
    def find_classes(self, directory: str):
        classes = ["colon_image_sets/colon_aca","colon_image_sets/colon_n","lung_image_sets/lung_aca","lung_image_sets/lung_n","lung_image_sets/lung_scc"]
        return classes, {val:ind for ind, val in enumerate(classes)}

In [34]:
lung_colon_dataset = LungColonDS(BASE,torchvision.io.read_image, extensions = tuple([".jpeg"]))

# Making Dataloader
Dataloaders is how we will present the data to the trainer, so we need to create this object. In the future, these can be adjusted to handle any kind of new distribution we want to make

In [35]:
from torch.utils.data import DataLoader

train_count = int(0.7*len(lung_colon_dataset))
valid_count = len(lung_colon_dataset) - train_count
train_dataset, valid_dataset = torch.utils.data.random_split(lung_colon_dataset, [train_count, valid_count])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [36]:
imgs, labels = next(iter(train_dataloader))

In [37]:
labels.shape

torch.Size([64])

In [38]:
imgs[0].shape

torch.Size([3, 768, 768])

# Training Resnet With Rotated Images

In [39]:
import torch
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt

In [40]:
BASE = "lung_colon_image_set/"

In [41]:
class LungColonDS(torchvision.datasets.DatasetFolder):
    def find_classes(self, directory: str):
        classes = ["colon_image_sets/colon_aca","colon_image_sets/colon_n","lung_image_sets/lung_aca","lung_image_sets/lung_n","lung_image_sets/lung_scc"]
        return classes, {val:ind for ind, val in enumerate(classes)}

In [42]:
lung_colon_dataset = LungColonDS(BASE,torchvision.io.read_image, extensions = (".jpeg",))
import RotatedDataset
lung_colon_rotated_dataset = RotatedDataset.RotatedDataset(lung_colon_dataset, use_both_labels = True)

In [43]:
lung_colon_dataset.classes

['colon_image_sets/colon_aca',
 'colon_image_sets/colon_n',
 'lung_image_sets/lung_aca',
 'lung_image_sets/lung_n',
 'lung_image_sets/lung_scc']

In [44]:
train_count = int(0.7*len(lung_colon_rotated_dataset))
valid_count = len(lung_colon_rotated_dataset) - train_count
train_dataset, valid_dataset = torch.utils.data.random_split(lung_colon_rotated_dataset, [train_count, valid_count])

In [45]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [46]:
import resnet
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [47]:
learning_rate = 2.5e-4
model = resnet.ResNet.TTT_Implementation(5,4,True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

TypeError: __init__() missing 3 required positional arguments: 'F', 'A', and 'B'

In [37]:
def check_accuracy_TTT(loader, model):
    #Only check accuracy of main task
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y[0]
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)[0]
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples if float(num_correct) != 0 else 0
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

def trainTTT(model, optimizer, epochs=1):
    """
    Train a TTT model.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: The accuracy of the model
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_dataloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y0,y1 = y
            y0 = y0.to(device=device, dtype=torch.long)
            y1 = y1.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores[0], y0) + F.cross_entropy(scores[1], y1)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if (t + 1) % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t + 1, loss.item()))
                check_accuracy_part34(test_dataloader, model)
                print()
    return check_accuracy_TTT(test_dataloader, model)

In [38]:
train_dataloader

In [39]:
print_every = 100
train_part34(model, optimizer, epochs=10)
print_every = 100

Epoch 0, Iteration 100, loss = 0.4048
Got 6457 / 7500 correct (86.09)

Epoch 0, Iteration 200, loss = 0.3017
Got 4587 / 7500 correct (61.16)

Epoch 1, Iteration 100, loss = 0.4193
Got 4644 / 7500 correct (61.92)

Epoch 1, Iteration 200, loss = 0.3993
Got 5741 / 7500 correct (76.55)

Epoch 2, Iteration 100, loss = 0.3307
Got 5178 / 7500 correct (69.04)

Epoch 2, Iteration 200, loss = 0.2225
Got 5908 / 7500 correct (78.77)

Epoch 3, Iteration 100, loss = 0.1598
Got 5618 / 7500 correct (74.91)

Epoch 3, Iteration 200, loss = 0.2475
Got 5810 / 7500 correct (77.47)

Epoch 4, Iteration 100, loss = 0.1837
Got 4425 / 7500 correct (59.00)

Epoch 4, Iteration 200, loss = 0.1140
Got 4605 / 7500 correct (61.40)

Epoch 5, Iteration 100, loss = 0.1617
Got 5994 / 7500 correct (79.92)

Epoch 5, Iteration 200, loss = 0.2732
Got 6871 / 7500 correct (91.61)

Epoch 6, Iteration 100, loss = 0.1792
Got 5832 / 7500 correct (77.76)

Epoch 6, Iteration 200, loss = 0.1539
Got 6192 / 7500 correct (82.56)

Epoch 